In [ ]:
import os
import io
import sys 
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import ollama
from IPython.display import Markdown, display, update_display
import gradio as gr

In [ ]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [ ]:
openai = OpenAI()
OPENAI_MODEL = "gpt-4o-mini"
LLAMA_MODEL = "llama3.1:8b"
QWEN_MODEL = "qwen2.5-coder:14b"

In [ ]:
# Uses Llama to Check Which Language the Code is Written In
system_message_comments = "You are an assistant designed to add docstrings and helpful comments to code for documentation purposes."
system_message_comments += "Respond back with properly formatted code, including docstrings and comments. Keep comments concise. "
system_message_comments += "Do not respond with greetings, or any such extra output"

In [ ]:
system_message_summary = "You are an assistant designed to summarise code for documentation purposes. You are not to display code again."
system_message_summary += "Respond back with a properly crafted summary, mentioning key details regarding to the code, such as workflow, code language."
system_message_summary += "Do not respond with greetings, or any such extra output. Do not respond in Markdown. Be thorough, keep explanation level at undergraduate level."

In [ ]:
def user_prompt_for(code):
    user_prompt = "Rewrite this code to include helpful comments and docstrings. "
    user_prompt += "Respond only with code.\n"
    user_prompt += code
    return user_prompt

In [ ]:
def user_prompt_for_summary(code):
    user_prompt = "Return the summary of the code.\n"
    user_prompt += code
    return user_prompt

In [ ]:
def messages_for(code):
    messages = [
        {"role": "system", "content": system_message_comments},
        {"role":"user", "content" : user_prompt_for(code)}
    ]
    return messages

In [ ]:
def messages_for_summary(code):
    messages = [
        {"role": "system", "content": system_message_summary},
        {"role":"user", "content" : user_prompt_for_summary(code)}
    ]
    return messages

In [ ]:
func = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
def call_llama(code):
    messages = messages_for(code)
    response1 = ollama.chat(model=LLAMA_MODEL, messages=messages)

    # summary
    messages = messages_for_summary(code)
    response2 = ollama.chat(model=LLAMA_MODEL, messages=messages)
    
    return response1['message']['content'],response2['message']['content']

In [ ]:
def call_qwen(code):
    # commented code
    messages = messages_for(code)
    response1 = ollama.chat(model=QWEN_MODEL, messages=messages)

    # summary
    messages = messages_for_summary(code)
    response2 = ollama.chat(model=QWEN_MODEL, messages=messages)
    
    return response1['message']['content'],response2['message']['content']

In [ ]:
def call_gpt(code):
    # commented code
    completion1 = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages_for(code),
    )

    #summary
    completion2 = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages_for_summary(code),
    )
    
    return completion1.choices[0].message.content,completion2.choices[0].message.content

In [ ]:
def create_docs(code,model):
    if model == "Llama":
        comments,summary = call_llama(code)
    elif model == "GPT":
        comments,summary = call_gpt(code)
    elif model == "QWEN":
        comments,summary = call_qwen(code)
    else:
        raise ValueError("Unknown Model")
    return comments,summary

In [ ]:
css = """
.comments {background-color: #00599C;}
.summary {background-color: #008B8B;}
"""

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("### Code Documentation and Formatting")
    with gr.Row():
        code = gr.Textbox(label="Input Code: ", value=func, lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT","Llama","QWEN"],label="Select model",value="GPT")
    with gr.Row():
        docs = gr.Button("Add Comments and Sumarise Code")
    with gr.Row():
        commented_code = gr.Textbox(label= "Formatted Code", lines=10,elem_classes=["comments"])
        code_summary = gr.Textbox(label = "Code Summary", lines=10,elem_classes=["summary"])
    docs.click(create_docs,inputs=[code,model],outputs=[commented_code,code_summary])

In [ ]:
ui.launch(inbrowser=True)

In [ ]:
print(call_llama(func))